# Run local LLMs with Ollama and Langchain

- Install ollama locally https://github.com/ollama/ollama
- Test

`ollama pull llama3`

`ollama run llama3`

- Tvailable models for native download - https://ollama.com/library
- Can also download models from huggingface , notably quantized models
- Download gguf from huggingface, create a Modelfile, then e.g. 

`ollama create mixtral -f Modelfile`

- You can quantize and fine-tune your own
- https://www.answer.ai/posts/2024-03-06-fsdp-qlora.html
- This is run on M3 Macbook Air with 24GB memory


In [1]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta

import yaml

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory
from langchain.tools import StructuredTool
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain_core.tools import StructuredTool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# langchain is still on pydantic v1
import pydantic.v1
from pydantic.v1 import BaseModel, Field # <-- Uses v1 namespace

import wikipedia

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
page = 'Eli Lilly and Company'

def fn_get_wiki_page(page):
    """Given the name of existing wikipedia page, return the content of the page"""
    return wikipedia.page(title=page, auto_suggest=False).content

page_content = fn_get_wiki_page(page)
page_content[:1000]
print(len(page_content))


56222


In [3]:
prompts = [
"Translate ‘You are amazing and I love you’ to French.",
"Define ‘photosynthesis’.",
"Correct the grammar: ‘He go to the store yesterday.’",
"Write a humorous limerick about hedge funds where the first letter of each verse spells 'hedge'.",
f"""You will act as a securities analyst with deep knowledge of financial markets,
securities analysis, and portfolio management. You will maintain a professional yet engaging tone,
in the style of a senior investment bank research analyst. You will concisely provide facts
relevant to investors and avoid opinions unless specifically requested.

You will focus on Eli Lilly (LLY). You will summarize the provided content below from
Wikipedia for Eli Lilly and summarize all topics of interest to investors on this page in 3000
words or less: {page_content}""",
]
prompts

['Translate ‘You are amazing and I love you’ to French.',
 'Define ‘photosynthesis’.',
 'Correct the grammar: ‘He go to the store yesterday.’',
 "Write a humorous limerick about hedge funds where the first letter of each verse spells 'hedge'.",
 'You will act as a securities analyst with deep knowledge of financial markets,\nsecurities analysis, and portfolio management. You will maintain a professional yet engaging tone,\nin the style of a senior investment bank research analyst. You will concisely provide facts\nrelevant to investors and avoid opinions unless specifically requested.\n\nYou will focus on Eli Lilly (LLY). You will summarize the provided content below from\nWikipedia for Eli Lilly and summarize all topics of interest to investors on this page in 3000\nwords or less: Eli Lilly and Company is an American pharmaceutical company headquartered in Indianapolis, Indiana, with offices in 18 countries. Its products are sold in approximately 125 countries. The company was founded

In [19]:
models = [
#     "Yi_lm",
#     "gemma_lm",
#     "yi",
#     "aya",
    "phi3",
    "qwen2",
    "gemma2",
#     "llama3",
#     "mistral",
#     "mixtral",
]


In [20]:
local_llm = ChatOllama(model=models[0], temperature=0)

messages = [
    SystemMessage(content="You are a friendly assistant."),
    HumanMessage(content=prompts[0]),
]

response = local_llm.invoke(messages)

print(response.content, "\n")


 Vous êtes incroyable et j'aime vous. 



In [23]:
system_template = "You are a helpful assistant."

all_responses = ""

for m in models:
    
    print(f"# {m}\n")
    response = f"# {m}\n"
    local_llm = ChatOllama(model=m, temperature=0)
    
    for p in prompts:
        print(p[:200])

        prompt_template = ChatPromptTemplate.from_messages(
            [("system", system_template),
             ("user", p)]
        )

        parser = StrOutputParser()

        chain = prompt_template | local_llm | parser

        # stream tokens as they are generated
        for r in chain.stream({}):
            print(r, end="")
            response += r

        print("\n\n---\n\n")
        response += "\n\n---\n\n"
        
        
    all_responses += response
    all_responses += "\n\n---\n\n"
        
        
# display(Markdown(all_responses))

# response


# phi3

Translate ‘You are amazing and I love you’ to French.
 Tu es incroyable et je t'aime.

---


Define ‘photosynthesis’.
 Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose or other sugars. This complex biochemical reaction occurs primarily within specialized cell structures called chloroplasts found in plant cells.

During photosynthesis, plants use carbon dioxide (CO2) and water (H2O), along with sunlight energy, to produce glucose (C6H12O6) and oxygen gas (O2). The overall chemical equation for this process can be represented as:

6 CO2 + 6 H2O + light energy → C6H12O6 + 6 O2

Photosynthesis consists of two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle). In the first stage, sunlight is absorbed by chlorophyll pigments in the thylakoid membranes within the chloroplasts. This energy is used to split wat

Using the captured light energy, plants combine water and carbon dioxide to create glucose (sugar), which is their food source.  As a byproduct, they release oxygen into the atmosphere.

**In essence, photosynthesis is how plants make their own food and provide the oxygen we breathe!** 


Let me know if you'd like to know more about the specific steps involved in photosynthesis or its importance to life on Earth.

---


Correct the grammar: ‘He go to the store yesterday.’
He **went** to the store yesterday. 


---


Write a humorous limerick about hedge funds where the first letter of each verse spells 'hedge'.
**H**igh-rolling traders, sharp and keen,
**E**ager to make fortunes unseen.
**D**ealing in stocks and more,
**G**etting rich, that's for sure,
**E**ven if the market's a mean machine. 


Let me know if you'd like to try another limerick! 


---


You will act as a securities analyst with deep knowledge of financial markets,
securities analysis, and portfolio management. You wil

In [24]:
display(Markdown(all_responses))


# phi3
 Tu es incroyable et je t'aime. Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose or other sugars. This complex biochemical reaction occurs primarily within specialized cell structures called chloroplasts found in plant cells.

During photosynthesis, plants use carbon dioxide (CO2) and water (H2O), along with sunlight energy, to produce glucose (C6H12O6) and oxygen gas (O2). The overall chemical equation for this process can be represented as:

6 CO2 + 6 H2O + light energy → C6H12O6 + 6 O2

Photosynthesis consists of two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle). In the first stage, sunlight is absorbed by chlorophyll pigments in the thylakoid membranes within the chloroplasts. This energy is used to split water molecules into hydrogen ions and oxygen gas, while also generating ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate), which are essential for the second stage of photosynthesis.

In the light-independent reactions or Calvin cycle, the energy stored in ATP and NADPH is used to convert carbon dioxide into glucose through a series of enzyme-catalyzed chemical reactions. The produced glucose can then be utilized by plants for growth, reproduction, and other metabolic processes or stored as starch for later use.

Photosynthesis is essential to life on Earth, as it provides the primary source of energy for most living organisms through food chains and contributes significantly to oxygen production in our atmosphere. He went to the store yesterday. Hedge funds, oh how they grow,
Eager investors in tow.
Dollars dance and sway,
Gains come out on display,
Everyone's wealth just starts to flow! Eli Lilly and Company, founded in 1876 by Colonel John E. Lilly, is a global biopharmaceecial company headquartered in Indianapolis, Indiana. It has grown to become one of the largest pharmaceutical companies worldwide, with its products spanning across various therapeutic areas including oncology, diabetes care, and neuroscience.

The company's history is marked by significant milestones such as the development of insulin in 1923, which revolutionized diabetes treatment. Eli Lilly also faced legal challenges over marketing practices, including an illegal promotion of Evista for conditions not approved by the FDA and a controversy surrounding rising insulin prices.

Eli Lilly's commitment to innovation is evident in its extensive research and development efforts, with numerous patents granted annually. The company has also been involved in various philanthropic activities, including supporting education through scholarships and grants.

Despite facing criticism for high drug prices, Eli Lilly continues to invest heavily in R&D, aiming to address unmet medical needs and improve patient outcomes. Its recent initiatives include reducing the cost of insulin and offering discounts on certain medications, reflecting a response to public concerns over healthcare affordability.

Eli Lilly's impact extends beyond its products; it has contributed significantly to Indiana's economy and cultural heritage, with historical landmarks like Building 314 in Indianapolis serving as reminders of the company's legacy.

---

# qwen2
"Vous êtes incroyable et je vous aime"Photosynthesis is the process used by plants, algae, and certain bacteria to convert light energy into chemical energy that can be used to fuel their growth. This process occurs in the chloroplasts of plant cells, which contain a green pigment called chlorophyll.

During photosynthesis, these organisms absorb carbon dioxide (CO2) from the air and water (H2O) from the soil using sunlight as an energy source. The light energy is captured by chlorophyll and used to convert CO2 and H2O into glucose (C6H12O6), a simple sugar that serves as food for the plant, and oxygen (O2), which is released into the air.

The overall chemical equation for photosynthesis can be represented as:

\[ 6CO_2 + 6H_2O + light \rightarrow C_6H_{12}O_6 + 6O_2 \]

This process not only provides energy for plants but also plays a crucial role in the global carbon cycle and is responsible for producing most of the oxygen found in Earth's atmosphere."He went to the store yesterday."There once was a fund quite bold,
Whose name started with "H" and gold,
In markets it roared,
With strategies galore,
But always kept folks on the hold.  

Its managers, so sly,
Would bet on what's up or low,
With leverage they'd play,
And portfolios they'd sway,
Yet, risks were not to be told.

In tales of its trade,
Some saw wealth in their stride,
While others would dread,
The volatility spread,
For "hedge" was a code for surprise.  

So here ends our rhyme,
Of funds that might climb or decline,
Remember the rule:
When investing, beware the fool,
And hedge your bets with care and line.Eli Lilly and Company is a global pharmaceutical company founded in Indianapolis, Indiana by Colonel Eli Lilly in 1876. The company's history includes significant contributions to the field of medicine through research and development of various drugs.

Key points about Eli Lilly:

- **Innovation**: Eli Lilly has been at the forefront of innovation with numerous breakthroughs including insulin production for diabetes management, antipsychotic medication Zyprexa, and cancer treatments like Evista. 

- **Legal Issues**: The company faced significant legal challenges over marketing practices, particularly concerning its drug Zyprexa which led to large settlements due to allegations of illegal promotion.

- **Patent Disputes**: Eli Lilly has also been involved in patent disputes including a lawsuit against Canada for alleged violations under the North American Free Trade Agreement regarding patents on drugs Strattera and Zyprexa.

- **Insulin Pricing Controversy**: In recent years, Eli Lilly faced criticism over insulin pricing, with accusations of price gouging leading to investigations by US lawmakers. 

- **Corporate Responsibility**: Despite these challenges, Eli Lilly has been recognized for its contributions to healthcare and has implemented measures to ensure integrity in business practices.

Eli Lilly continues to operate globally, contributing significantly to the pharmaceutical industry through research and development while facing ongoing scrutiny over its operations and practices.

---

# gemma2
"Tu es incroyable et je t'aime." 


Let me know if you'd like to translate anything else! 😊  
Photosynthesis is the amazing process used by plants, algae, and some bacteria to convert light energy from the sun into chemical energy in the form of sugars. 

Here's a simple breakdown:

* **Sunlight:** Plants capture light energy using a green pigment called chlorophyll.
* **Water:** Plants absorb water from the soil through their roots.
* **Carbon Dioxide:** Plants take in carbon dioxide from the air through tiny pores on their leaves called stomata.

Using the captured light energy, plants combine water and carbon dioxide to create glucose (sugar), which is their food source.  As a byproduct, they release oxygen into the atmosphere.

**In essence, photosynthesis is how plants make their own food and provide the oxygen we breathe!** 


Let me know if you'd like to know more about the specific steps involved in photosynthesis or its importance to life on Earth.He **went** to the store yesterday. 
**H**igh-rolling traders, sharp and keen,
**E**ager to make fortunes unseen.
**D**ealing in stocks and more,
**G**etting rich, that's for sure,
**E**ven if the market's a mean machine. 


Let me know if you'd like to try another limerick! 
This text provides a comprehensive overview of Eli Lilly and Company, covering its history, community service, controversies, and more. 

Here's a breakdown of the key points:

**History:**

* Founded in 1876 by Colonel Eli Lilly in Indianapolis, Indiana.
* Initially focused on producing morphine and other pharmaceutical products.
* Expanded its product line to include insulin, Prozac, Zyprexa, and other medications.
* Acquired by its current parent company in 1987.

**Community Service:**

* Long history of philanthropy and support for local organizations.
* Provided aid to disaster victims, including the San Francisco earthquake and the Johnstown Flood.
* Established Lilly Field Hospital 32 during World War I.
* Manufactured over 200 products for military use during World War II.

**Foundations:**

* **Lilly Endowment:** Founded in 1937 by Josiah K. Lilly Sr. and his sons, Eli and Joe.
* **Eli Lilly and Company Foundation:** Established in 1968, funded through corporate profits.

**Controversies:**

* **BGH:** Purchase of bovine growth hormone (BGH) from Monsanto in 2008, facing ethical and health concerns.
* **340B:** Legal battle with the Department of Health and Human Services over drug pricing for hospitals.
* **Prozac:** Lawsuits and public outcry following a 1989 mass shooting linked to the drug.
* **Zyprexa:** Multiple lawsuits and criminal charges for illegally marketing the drug for unapproved uses.
* **Discrimination:** Accused of sex discrimination and age discrimination in hiring practices.
* **Canada Patent Lawsuit:** Sued Canada for violating its obligations to foreign investors.
* **Evista:** Guilty plea and fine for illegally promoting the drug for unapproved uses.
* **Insulin Pricing:** Facing criticism for high insulin prices and slow response to calls for affordability.

**Overall:**

The text paints a complex picture of Eli Lilly and Company, highlighting its contributions to medicine and society alongside its involvement in controversies and legal battles. It demonstrates the challenges faced by pharmaceutical companies in balancing innovation, profitability, and ethical considerations.




---

